# BasicTick: Create DB

## Steps
1. Untar hdb.tar.gz for the hdb data
2. Upload hdb to staging S3 bucket
3. Create database


## Setup
imports, environmentId, accountId

In [ ]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env import *


In [ ]:
# Source data directory
SOURCE_DATA_DIR="hdb"

# S3 bucket for external data and code
S3_DEST=f"s3://{S3_BUCKET}/data/{SOURCE_DATA_DIR}/"

# Managed KX Database and Cluster names to create
DB_NAME="basictickdb"

In [ ]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

## 1. Untar hdb.tar.gz
hdb database will be found in hdb directory

In [ ]:
!tar -xf hdb.tar.gz

In [ ]:
!ls -la hdb

# 2. Upload hdb data
using aws cli, copy hdb to staging bucket

In [ ]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
else:
    cp = f"""
aws s3 sync  --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
    
# execute the S3 copy
os.system(cp)

## 3. Create database

In [ ]:
# assume it exists
create_db=False

try:
    resp = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
    resp.pop('ResponseMetadata', None)
except:
    # does not exist, will create
    create_db=True

if create_db:
    print(f"CREATING Database: {DB_NAME}")
    resp = client.create_kx_database(environmentId=ENV_ID, databaseName=DB_NAME, description="Basictick kdb database")
    resp.pop('ResponseMetadata', None)

    print(f"CREATED Database: {DB_NAME}")

print(json.dumps(resp,sort_keys=True,indent=4,default=str))

## 4. Add HDB data to database

In [ ]:
changes=[]

for f in os.listdir("hdb"):
    if os.path.isdir(f"hdb/{f}"):
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}/", 'dbPath': f"/{f}/" } )
    else:
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}", 'dbPath': f"/" } )
        
resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, 
    changeRequests=changes)

resp.pop('ResponseMetadata', None)
changeset_id = resp['changesetId']

print("Changeset...")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))

In [ ]:
wait_for_changeset_status(client, environmentId=ENV_ID, databaseName=DB_NAME, changesetId=changeset_id)
print("**Done**")

In [ ]:
note_str = ""

c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)

if len(c_set_list) == 0:
    note_str = "<<Could not get changesets>>"
    
print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"  Changeset: {c_set_id}: Created: {c['createdTimestamp']} ({c['status']})")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )